In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json', 'r'))

In [ ]:
len(cv_data)

200

In [ ]:
# !pip install spacy_transformers

In [ ]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [3]:
!pip install spacy_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.5/193.5 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.2 MB/s eta 0:00:00


In [ ]:
# !nvidia-smi

In [ ]:
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db
      

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [ ]:
len(train), len(test)

(140, 60)

In [ ]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 60/60 [00:00<00:00, 84.20it/s]


In [ ]:
len(db.tokens)

60

In [ ]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-27 12:46:15,314] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-27 12:46:15,326] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-12-27 12:46:15,329] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-27 12:46:15,331] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 514kB/s]
Downloading: 100% 899k/899k [00:00<00:00, 2.09MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 1.06MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 2.61MB/s]
Downloading: 100% 501M/501M [00:05<00:00, 90.7MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'l

In [4]:
import spacy_transformers



In [5]:
nlp =spacy.load("/content/drive/MyDrive/output/model-best")

In [6]:
# doc = nlp('my name is Mohit rajesh singh. I worked at Microsoft and facebook. I have 10 years of experience.my mail is mohitsinghx8286@gmail.com')
# for ent in doc.ents:
#   print(ent.text, "   ->>>>> ", ent.label_)

In [7]:
!pip install PyMuPDF
!pip install spacy_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 83.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import sys, fitz
import spacy_transformers
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json


In [ ]:
# nlp =spacy.load("/content/drive/MyDrive/output/model-best")
# import pickle
# filename = 'finalized_model.sav'
# pickle.dump(nlp, open(filename, 'wb'))

In [ ]:
# fname = '/content/drive/MyDrive/java-developer-resume-example.pdf'
# doc = fitz.open(fname)

In [ ]:
# text = " "
# for page in doc:
#   text = text + str(page.get_text())

In [ ]:
# text = text.strip()

In [ ]:
# text = ' '.join(text.split())

In [ ]:
# text

In [ ]:
# doc = nlp(text)
# for ent in doc.ents:
#   print(ent.text, "   ->>>>> ", ent.label_)

In [9]:
def resumeanalyze() :
  fname = '/content/drive/MyDrive/java-developer-resume-example.pdf'
  doc = fitz.open(fname) 

  text = " "
  for page in doc:
       text = text + str(page.get_text())
       text = text.strip()
       text = ' '.join(text.split())
       print("Text :",text)
       doc = nlp(text)
       for ent in doc.ents:
             print(ent.text, "   ->>>>> ", ent.label_)
resumeanalyze()



Text : NADIA DELGADO Java Developer nadiadel@email.com (123) 456-7890 Detroit, MI linkedin.com/in/nadia-del github.com/nadia-del EDUCATION B.S. Computer Science University of Michigan September 2011 - June 2015 Ann Arbor, MI SKILLS Java JavaScript Angular.js HTML CSS UNIX SQL Eclipse Oracle React.js CAREER OBJECTIVE Experienced Java Developer that dreams in Java and has a solid grasp on web services development and complex application development. I am excited to continue my career by joining a company composed of driven professionals who value creative problem solving and strive to revolutionize Centria's healthcare. WORK EXPERIENCE Java Developer Deloitte May 2018 - current Lansing, MI · Designed software solutions by analyzing system performance standards, increasing performance efﬁciency by 27%. · Analyzed systems ﬂow, data usage, and work processes, and investigated problem areas. · Planned, tracked, and managed deliverables on short-term sprints and long-term software deployments

In [ ]:
# import joblib
# loaded_model = joblib.load(filename)
# def resumeanalyze() :
#   fname = '/content/drive/MyDrive/java-developer-resume-example.pdf'
#   doc = fitz.open(fname) 

#   text = " "
#   for page in doc:
#        text = text + str(page.get_text())
#        text = text.strip()
#        text = ' '.join(text.split())
#        print("Text :",text)
#        doc = loaded_model(text)
#        for ent in doc.ents:
#              print(ent.text, "   ->>>>> ", ent.label_)
# resumeanalyze()



In [10]:

!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 7.2 MB/s eta 0:00:00


In [ ]:
!./ngrok authtokens 2JuX65ABZVQuWkylJPNZxEKCr3S_6LTvc8JjXMDLVcomKQwqk

/bin/bash: ./ngrok: No such file or directory


In [12]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=e3e3a381e61a84e5c99b4b245fc4e6763a89ac6c11afe949e62da8ec010475cb
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [13]:
from pyngrok import ngrok 
public_url = ngrok.connect(port='8501')
public_url

<NgrokTunnel: "http://bfb2-34-91-103-15.ngrok.io" -> "http://localhost:80">

In [14]:
!pip install PyPDF3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 KB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyPDF3: filename=PyPDF3-1.0.6-py3-none-any.whl size=62882 sha256=6ba35c4dedd7127a9eef9ab140b039e4a8d0fbc6755c9d71f4a680fb6b624a76
  Stored in directory: /root/.cache/pip/wheels/78/ca/e9/8f361b015e0b94fe66389b176ab4aa03a8875f9c7ff22ac397
Successfully built PyPDF3


In [15]:
# pip install tensorflow==2.9.1
# !export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python
!pip install protobuf==3.20.*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
# !pip uninstall protobuf
# !pip install protobuf

In [ ]:
# !pip install falcon
# !pip install pdfminer3
# !pip install pafy
# !pip install plotly
# !pip install pymysql
# !pip install streamlit-tags
# !pip install Pillow
# !pip install ResumeParser
# !pip install nltk

In [16]:
%%writefile streamlit_app1.py 

import spacy
import streamlit as st
from PyPDF3 import PdfFileReader

# Load the Spacy model for resume parsing
nlp =spacy.load("/content/drive/MyDrive/output/model-best")

def extract_info(resume_text):
    # Process the resume text with Spacy
    doc = nlp(resume_text)
    
    # Extract information from the resume
    education = []
    work_experience = []
    skills = []
    Name = []
    Designation = []
    Location = []
    Companies_worked_at = []
    Email_Address = []
    college_name = []
    for ent in doc.ents:
        if ent.label_ == "Degree":
            education.append(ent.text)
        elif ent.label_ == "Years of Experience":
            work_experience.append(ent.text)
        elif ent.label_ == "Skills":
            skills.append(ent.text)
        elif ent.label_ == "Name":
            Name.append(ent.text) 
        elif ent.label_ == "Companies worked at":
            Companies_worked_at.append(ent.text)  
        elif ent.label_ == "Location":
            Location.append(ent.text) 
        elif ent.label_ == "Designation":
            Location.append(ent.text) 
        elif ent.label_ == "Email Address":
            Location.append(ent.text)   
        elif ent.label_ == "college name":
            Location.append(ent.text)                       

    return education, work_experience, skills, Name, Location, Companies_worked_at, Designation, Email_Address, college_name

def main():
    st.set_page_config(page_title="Resume Parser", page_icon=":guardsman:", layout="wide")
    st.title("Resume Parser")
    
    # Get the resume text from the user
   # resume_text = st.text_area("Paste your resume text here")
    pdf_file = st.file_uploader("Upload a PDF file", type=["pdf"])
    if pdf_file is not None:
        pdf_reader = PdfFileReader(pdf_file)
        resume_text = pdf_reader.getPage(0).extractText()
        st.write(resume_text)

    
    # Extract information from the resume
    if st.button("Extract"):
        education, work_experience, skills, Name, Designation, Location, Companies_worked_at, Email_Address, college_name = extract_info(resume_text)
        
        # Display the results
        st.header("Education")
        for edu in education:
            st.write("- " + edu)
        st.header("Work Experience")
        for exp in work_experience:
            st.write("- " + exp)
        st.header("Skills")
        for skill in skills:
            st.write("- " + skill)
        st.header("Name")
        for name in Name:
            st.write("- " + name)
        st.header("Location")
        for location in Location:
            st.write("- " + location)
        st.header("Companies Worked")
        for cwa in Companies_worked_at:
            st.write("- " + cwa)
        st.header("Designation")
        for desg in Designation:
            st.write("- " + desg)            
        st.header("Contacts")
        for email in Email_Address:
            st.write("- " + email)     
        st.header("college name")
        for clg in college_name:
            st.write("- " + clg)                   

if __name__ == "__main__":
    main()



Writing streamlit_app1.py


In [ ]:
!streamlit run /content/streamlit_app1.py & npx localtunnel --port 8501

[..................] / rollbackFailedOptional: verb npm-session 9a09a0c54788778

npx: installed 22 in 4.196s
your url is: https://twenty-wolves-flash-34-91-103-15.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.91.103.15:8501



In [ ]:
%%writefile streamlit_app2.py 

import streamlit as st
from PyPDF3 import PdfFileReader

pdf_file = st.file_uploader("Upload a PDF file", type=["pdf"])

if pdf_file is not None:
    pdf_reader = PdfFileReader(pdf_file)
    pdf_text = pdf_reader.getPage(0).extractText()
    st.write(pdf_text)






Overwriting streamlit_app2.py


In [ ]:
!streamlit run /content/streamlit_app2.py & npx localtunnel --port 8501



npx: installed 22 in 5.044s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.240.137.168:8501

your url is: https://eight-gifts-think-35-240-137-168.loca.lt
  Stopping...
^C


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 KB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyPDF3: filename=PyPDF3-1.0.6-py3-none-any.whl size=62882 sha256=283ea056ba229cdce6c74262bdf7d83118d2b56642e889029ca963ce45fbeb00
  Stored in directory: /root/.cache/pip/wheels/78/ca/e9/8f361b015e0b94fe66389b176ab4aa03a8875f9c7ff22ac397
Successfully built PyPDF3
